In [1]:
import os
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from dataset import load_core_set_data, load_pca_data
from simulation.simulator import FDTDSimulator
from simulation.student import LSTMPredictor

In [2]:
radius_save_path = "data/radius_matrix.pth"

if os.path.exists(radius_save_path):
    radius_matrix = torch.load(radius_save_path)
else:
    radius_matrix = torch.rand(10, 10) * 10
    torch.save(radius_matrix, radius_save_path)

print(radius_matrix)


tensor([[4.2537, 7.2491, 6.1717, 9.2729, 4.9853, 9.8473, 3.1820, 9.3614, 8.1104,
         9.0288],
        [2.3149, 8.7649, 5.0795, 8.0234, 4.8227, 8.7122, 0.7336, 2.7354, 7.6845,
         6.5359],
        [6.5778, 9.8060, 0.8749, 7.8437, 6.0056, 7.5152, 8.9349, 6.0827, 4.3046,
         7.7731],
        [0.3096, 1.4307, 0.4254, 2.0687, 3.2475, 0.8592, 9.9883, 3.1064, 7.1883,
         2.8260],
        [6.0670, 4.3900, 8.1416, 5.1876, 1.6904, 2.8222, 9.9509, 8.0686, 6.5593,
         5.9664],
        [9.7981, 5.3376, 9.5862, 0.3684, 5.5181, 7.1683, 3.2136, 5.2784, 1.8340,
         9.4031],
        [2.1909, 0.4959, 9.4064, 1.8535, 2.7756, 7.6035, 9.4730, 6.0837, 7.2479,
         1.7767],
        [2.4067, 8.6760, 5.0395, 6.0531, 6.9528, 0.0580, 0.6878, 2.1934, 4.4764,
         4.8344],
        [4.9628, 6.8850, 0.1238, 5.7276, 2.3660, 0.6239, 9.2926, 6.6920, 3.6652,
         1.5679],
        [2.8365, 5.7207, 1.2948, 2.9172, 4.4617, 8.5791, 4.1536, 8.3898, 6.0025,
         7.9040]])


C:\Users\xincy\AppData\Local\Temp\ipykernel_2120\835076115.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  radius_matrix = torch.load(radius_save_path)


In [3]:
class FDTDDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.FloatTensor(data)
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


train_data, _, test_data, _ = load_core_set_data()

print(train_data.shape)
print(test_data.shape)

simulator = FDTDSimulator(radius_matrix=radius_matrix)

train_labels = torch.tensor([])
for data in train_data:
    inputs = torch.tensor(data)
    train_labels = torch.cat((train_labels, simulator(inputs)), dim=0)

print(train_labels.shape)

train_dataset = FDTDDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_labels = torch.tensor([])
for data in test_data:
    inputs = torch.tensor(data)
    test_labels = torch.cat((test_labels, simulator(inputs)), dim=0)

test_dataset = FDTDDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


(300, 10)
(50, 10)


KeyboardInterrupt: 

In [ ]:
train_dataset_save_path = "data/train_dataset.pth"
test_dataset_save_path = "data/test_dataset.pth"

torch.save(train_dataset, train_dataset_save_path)
torch.save(test_dataset, test_dataset_save_path)
